In [1]:
import json
from StringIO import StringIO
import pandas as pd

In [2]:
import os
import sklearn as sk
from sklearn.linear_model import MultiTaskLasso, MultiTaskElasticNet, ElasticNet
from sklearn.cross_validation import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
import sklearn.metrics as skmet
import numpy as np
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords, names
import datetime
from scipy.stats import ttest_ind 
import csv

In [3]:
listings = pd.read_csv('inside_airbnb/listings.csv')
list_summ = pd.read_csv('inside_airbnb/listings_summ.csv')
neighborhoods = pd.read_csv('inside_airbnb/neighbourhoods.csv')
reviews = pd.read_csv('inside_airbnb/reviews.csv')
reviews_summ = pd.read_csv('inside_airbnb/reviews_summ.csv')
calendar = pd.read_csv('inside_airbnb/calendar.csv')

In [4]:
def string_to_set(x):
    c = set()
    for w in x[1:-1].split(","):
        c.add(w)
        
    return c

def has_amenity(x, amen_):
    if amen_ in x:
        return 1
    pass

In [5]:
a = listings['amenities'].fillna('{}')

listings['amenities_set'] = a.map(string_to_set)
all_amenities = set()
    
for idx in listings['amenities_set'].index:
    all_amenities = all_amenities.union(listings['amenities_set'][idx])

In [6]:
b = set(['', '"translation missing: en.hosting_amenity_49"','"translation missing: en.hosting_amenity_50"'])

In [7]:
all_amenities = all_amenities.difference(b)

In [8]:
def boolean_convert(z):
    if z:
        return 1
    else:
        return 0

In [9]:
listings['amenities_set'].map(lambda x: '"24-Hour Check-in"' in x).map(boolean_convert)

0       0
1       0
2       0
3       1
4       1
5       0
6       0
7       0
8       0
9       1
10      0
11      1
12      0
13      0
14      0
15      0
16      0
17      0
18      0
19      0
20      0
21      0
22      1
23      0
24      0
25      0
26      0
27      1
28      0
29      0
       ..
6578    0
6579    0
6580    0
6581    0
6582    0
6583    0
6584    0
6585    1
6586    0
6587    0
6588    0
6589    1
6590    0
6591    0
6592    0
6593    0
6594    1
6595    1
6596    0
6597    0
6598    0
6599    0
6600    0
6601    0
6602    1
6603    0
6604    0
6605    0
6606    1
6607    0
Name: amenities_set, dtype: int64

In [10]:
has_amenities_list = []
for amen in all_amenities:
    listings['has_' + amen] = listings['amenities_set'].map(lambda x: amen in x).map(boolean_convert)
    has_amenities_list.append('has_' + amen)

In [11]:
listings[has_amenities_list].isnull().sum()

has_"Other pet(s)"                  0
has_Essentials                      0
has_"Carbon Monoxide Detector"      0
has_"Elevator in Building"          0
has_"Indoor Fireplace"              0
has_Internet                        0
has_Washer                          0
has_Hangers                         0
has_"Buzzer/Wireless Intercom"      0
has_TV                              0
has_Gym                             0
has_"Fire Extinguisher"             0
has_"Hot Tub"                       0
has_"Cable TV"                      0
has_"Air Conditioning"              0
has_"Suitable for Events"           0
has_Kitchen                         0
has_"Family/Kid Friendly"           0
has_Shampoo                         0
has_Heating                         0
has_"Hair Dryer"                    0
has_Cat(s)                          0
has_"Smoke Detector"                0
has_Iron                            0
has_"Free Parking on Premises"      0
has_"Pets live on this property"    0
has_"Safety 

In [12]:
listings[has_amenities_list].sum()

has_"Other pet(s)"                    71
has_Essentials                      5547
has_"Carbon Monoxide Detector"      4170
has_"Elevator in Building"          1063
has_"Indoor Fireplace"              1695
has_Internet                        4756
has_Washer                          5192
has_Hangers                         3414
has_"Buzzer/Wireless Intercom"       338
has_TV                              5274
has_Gym                              974
has_"Fire Extinguisher"             3114
has_"Hot Tub"                       1458
has_"Cable TV"                      3365
has_"Air Conditioning"              2914
has_"Suitable for Events"            639
has_Kitchen                         6088
has_"Family/Kid Friendly"           4140
has_Shampoo                         4364
has_Heating                         5193
has_"Hair Dryer"                    3008
has_Cat(s)                           339
has_"Smoke Detector"                5385
has_Iron                            2915
has_"Free Parkin

In [13]:
a = listings['amenities'].fillna('{}')

listings['amenities_set'] = a.map(string_to_set)
all_amenities = set()
    
for idx in listings['amenities_set'].index:
    all_amenities = all_amenities.union(listings['amenities_set'][idx])
    
b = set(['', '"translation missing: en.hosting_amenity_49"','"translation missing: en.hosting_amenity_50"'])

for amen in all_amenities:
    
    if len(amen.split(' ')) == 1:
        listings['has_' + amen] = listings['amenities_set'].map(lambda x: amen in x).map(boolean_convert)
        continue
        
    if "" in amen:
        amen = amen[1:-1].replace(' ', '_')
            
    listings['has_' + amen] = listings['amenities_set'].map(lambda x: amen in x).map(boolean_convert)
    


In [14]:
a = listings['host_verifications'].map(lambda x: x[1:-1]).map(lambda j: j.split(',')).map(lambda k: set(k))
all_host_verifications = set()
    
for w in a.index:
    all_host_verifications = all_host_verifications.union(a[w])

In [15]:
for w in all_host_verifications:
    
    b = a.map(lambda x: w in x).map(boolean_convert)
    
    if '' in w:
        w = w.strip()[1:-1].replace(' ', '_')
            
    listings['uses_' + w] = b
    

In [16]:
uses_verification_list = []
for veri in all_host_verifications:
    
    if '' in veri:
        veri = veri.strip()[1:-1].replace(' ', '_')
            
    uses_verification_list.append('uses_' + veri)

In [17]:
listings[uses_verification_list]

,uses_phone,uses_linkedin,uses_email,uses_manual_online,uses_facebook,uses_amex,uses_phone,uses_sent_id,uses_jumio,uses_google,uses_weibo,uses_manual_offline,uses_kba,uses_kba,uses_reviews
0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1
1,0,0,1,1,0,0,0,0,1,0,0,0,0,0,1
2,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1
3,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1
4,0,0,1,0,1,0,0,0,0,0,0,0,1,1,1
5,0,0,1,0,1,0,0,0,0,0,0,0,1,1,1
6,0,0,1,0,0,0,0,0,0,0,0,0,1,1,1
7,0,0,1,0,0,0,0,0,0,0,0,0,1,1,1
8,0,0,1,0,0,0,0,0,0,0,0,0,1,1,1
9,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1


In [18]:
listings[uses_verification_list].isnull().sum()

uses_phone             0
uses_linkedin          0
uses_email             0
uses_manual_online     0
uses_facebook          0
uses_amex              0
uses_phone             0
uses_sent_id           0
uses_jumio             0
uses_google            0
uses_weibo             0
uses_manual_offline    0
uses_kba               0
uses_kba               0
uses_reviews           0
dtype: int64

In [19]:
listings[uses_verification_list].sum()

uses_phone              186
uses_linkedin           113
uses_email             6421
uses_manual_online      102
uses_facebook          1354
uses_amex                68
uses_phone              186
uses_sent_id              7
uses_jumio             2211
uses_google             166
uses_weibo                1
uses_manual_offline     128
uses_kba               2756
uses_kba               2756
uses_reviews           5706
dtype: int64

In [20]:
a = listings['host_verifications'].map(lambda x: x[1:-1]).map(lambda j: j.split(',')).map(lambda k: set(k))
all_host_verifications = set()
    
for w in a.index:
    all_host_verifications = all_host_verifications.union(a[w])
    
for w in all_host_verifications:
    
    b = a.map(lambda x: w in x).map(boolean_convert)
    
    if '' in w:
        w = w.strip()[1:-1].replace(' ', '_')
            
    listings['uses_' + w] = b
    

In [23]:
listings[listings['uses_weibo'] == 1]

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,uses_manual_online,uses_facebook,uses_amex,uses_sent_id,uses_jumio,uses_google,uses_weibo,uses_manual_offline,uses_kba,uses_reviews
6246,5523515,https://www.airbnb.com/rooms/5523515,20160706203047,2016-07-07,Sunny Room/Private Bath in UTC UCSD,"You will fall in love - UTC, UCSD. A walkable ...",LOCATION! LOCATION! LOCATION! Remodeled SUPER ...,"You will fall in love - UTC, UCSD. A walkable ...",none,"The University Towne Centre (UTC) area, locate...",...,0,0,0,0,1,0,1,0,0,1


In [25]:
listings["calendar_updated" ].unique()

array(['2 weeks ago', '11 months ago', 'a week ago', '4 weeks ago',
       '5 months ago', '5 weeks ago', 'today', '6 weeks ago',
       '2 months ago', '3 months ago', 'yesterday', '3 days ago',
       '3 weeks ago', '2 days ago', '4 days ago', '6 months ago', 'never',
       '9 months ago', '10 months ago', '5 days ago', '7 months ago',
       '13 months ago', '6 days ago', '22 months ago', '4 months ago',
       '16 months ago', '24 months ago', '1 week ago', '12 months ago',
       '7 weeks ago', '8 months ago', '23 months ago', '15 months ago',
       '18 months ago', '14 months ago', '17 months ago', '25 months ago',
       '41 months ago', '21 months ago', '28 months ago', '20 months ago',
       '19 months ago', '27 months ago'], dtype=object)

In [26]:
listings[listings["calendar_updated"]  == 'never']

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,uses_manual_online,uses_facebook,uses_amex,uses_sent_id,uses_jumio,uses_google,uses_weibo,uses_manual_offline,uses_kba,uses_reviews
30,13507453,https://www.airbnb.com/rooms/13507453,20160706203047,2016-07-07,Private master room & bathroom,Townhouse in a nice Neighborhood 1-big master ...,The room had 3 closet where you can put all yo...,Townhouse in a nice Neighborhood 1-big master ...,none,Quiet neighbors where you feel safe and enjoy ...,...,0,0,0,0,0,0,0,0,0,1
34,13559442,https://www.airbnb.com/rooms/13559442,20160706203047,2016-07-07,Private bedroom and bathroom,"A private room in a Nice condo, very quiet nei...","Quiet neighbours, the room has a big closet.","A private room in a Nice condo, very quiet nei...",none,NaN,...,0,0,0,0,0,0,0,0,0,1
49,13817382,https://www.airbnb.com/rooms/13817382,20160706203047,2016-07-07,Master room in chula vista!,Very beautiful master room with a huge queen s...,I live by myself and i'm not always at home,Very beautiful master room with a huge queen s...,none,The atmosphere! You feel as if you were in you...,...,0,0,0,0,0,0,0,0,0,1
73,2296907,https://www.airbnb.com/rooms/2296907,20160706203047,2016-07-07,Apartment Near Ocean,Enjoy a night of relaxation within 10 minute w...,NaN,Enjoy a night of relaxation within 10 minute w...,none,NaN,...,0,0,0,0,0,0,0,0,0,1
113,10394457,https://www.airbnb.com/rooms/10394457,20160706203047,2016-07-07,One bedroom and bathroom 12x10 room,Close to downtown and mission valley. Great ar...,NaN,Close to downtown and mission valley. Great ar...,none,NaN,...,0,0,0,0,0,0,0,0,0,0
268,13775219,https://www.airbnb.com/rooms/13775219,20160706203047,2016-07-07,Charming and comfortable 1 bed ap,My apartment has everything you need for a com...,NaN,My apartment has everything you need for a com...,none,NaN,...,0,1,0,0,1,0,0,0,0,1
317,9751948,https://www.airbnb.com/rooms/9751948,20160706203047,2016-07-07,Room 4 rent Chula Vista,"I have a room with furniture, tv, wiffi just o...",NaN,"I have a room with furniture, tv, wiffi just o...",none,One block from public transport and freeways,...,0,0,0,0,0,0,0,0,0,0
330,12149022,https://www.airbnb.com/rooms/12149022,20160706203047,2016-07-07,Peaceful and Comfy,"Comfortable and spacious, our home is located ...",The room is a 10x11 room not including the ful...,"Comfortable and spacious, our home is located ...",none,NaN,...,0,0,0,0,0,0,0,0,1,1
339,13238793,https://www.airbnb.com/rooms/13238793,20160706203047,2016-07-07,Beautiful luxurious room,Beautiful luxurious room in sunny San Diego. L...,We make sure to make your stay as perfect as p...,Beautiful luxurious room in sunny San Diego. L...,none,It's a quiet upscale neighborhood in Clairemon...,...,0,0,0,0,0,0,0,0,0,1
388,13789226,https://www.airbnb.com/rooms/13789226,20160706203047,2016-07-07,"Great Home, Perfect Location",Laid back easy going young professional lookin...,"The location is perfect, very cool AC!",Laid back easy going young professional lookin...,none,NaN,...,0,1,0,0,0,0,0,0,0,0


In [40]:
def calendar_update_parse(x):
    if x == 'today':
        return 0
    
    elif x == 'yesterday':
        return 1
    
    else:
        
        b = x.split(' ')
        
        if 'days' in b:
            a = int(b[0])
            return a
        
        elif 'week' in b:
            return 7
        
        elif 'weeks' in b:
            a = 7*int(b[0])
            return a
        
        elif 'month' in b:
            a = 365.25/12.0
            return np.floor(a)
        
        elif 'months' in b:
            a = (365.25*float(b[0]))/12.0
            return np.floor(a)
    pass
            

In [43]:
listings["calendar_updated" ].map(calendar_update_parse)

0        14.0
1        14.0
2       334.0
3         7.0
4        28.0
5        28.0
6       152.0
7        35.0
8       152.0
9         0.0
10       42.0
11       35.0
12       60.0
13       91.0
14        1.0
15        1.0
16        1.0
17       91.0
18        1.0
19        0.0
20        7.0
21        3.0
22        7.0
23       21.0
24        2.0
25        4.0
26        2.0
27      182.0
28        1.0
29       21.0
        ...  
6578      7.0
6579      3.0
6580      1.0
6581      1.0
6582      7.0
6583      0.0
6584      NaN
6585      0.0
6586      0.0
6587      2.0
6588      2.0
6589      7.0
6590      0.0
6591      0.0
6592      0.0
6593      0.0
6594      7.0
6595     14.0
6596      2.0
6597      1.0
6598      1.0
6599     28.0
6600     28.0
6601      7.0
6602     14.0
6603      7.0
6604     60.0
6605      5.0
6606      4.0
6607     21.0
Name: calendar_updated, dtype: float64

In [44]:
a = listings["calendar_updated" ].map(calendar_update_parse)

In [45]:
a = a.fillna(a.max())

In [46]:
a

0         14.0
1         14.0
2        334.0
3          7.0
4         28.0
5         28.0
6        152.0
7         35.0
8        152.0
9          0.0
10        42.0
11        35.0
12        60.0
13        91.0
14         1.0
15         1.0
16         1.0
17        91.0
18         1.0
19         0.0
20         7.0
21         3.0
22         7.0
23        21.0
24         2.0
25         4.0
26         2.0
27       182.0
28         1.0
29        21.0
         ...  
6578       7.0
6579       3.0
6580       1.0
6581       1.0
6582       7.0
6583       0.0
6584    1247.0
6585       0.0
6586       0.0
6587       2.0
6588       2.0
6589       7.0
6590       0.0
6591       0.0
6592       0.0
6593       0.0
6594       7.0
6595      14.0
6596       2.0
6597       1.0
6598       1.0
6599      28.0
6600      28.0
6601       7.0
6602      14.0
6603       7.0
6604      60.0
6605       5.0
6606       4.0
6607      21.0
Name: calendar_updated, dtype: float64